In [29]:
import pandas as pd
import requests
import json
import base64
import difflib

# df读取

In [2]:
df = pd.read_pickle("tensorflow-tensorflow-shas.tar.bz2")
df

,patch,sha,status,filename,parents_sha
0,"@@ -224,12 +224,7 @@ def __init__(self,\n ...",f21a440cde082e8b629f930c7c3f7b65ac27d4e6,modified,tensorflow/python/keras/callbacks.py,93d545e8b1e377926a24584a25c06dd422b7cc82
1,"@@ -274,6 +274,37 @@ def test_progbar_logging(...",f21a440cde082e8b629f930c7c3f7b65ac27d4e6,modified,tensorflow/python/keras/callbacks_test.py,93d545e8b1e377926a24584a25c06dd422b7cc82
2,"@@ -18,7 +18,6 @@\n from __future__ import pri...",7d9299129c30405270813fe32f1310dbf3bab265,modified,tensorflow/python/data/experimental/kernel_tes...,2c7d978a1446adaba32d8465aceef0b2c4a41cad
3,"@@ -0,0 +1,23 @@\n+# Copyright 2019 The Tensor...",23ec33ae1418d56c3ce0294720782b636595a2b3,added,tensorflow/python/keras/mixed_precision/experi...,ff513ac54689e57ed333d62e0399a4d64a536900
4,"@@ -33,7 +33,10 @@\n from tensorflow.python.ke...",23ec33ae1418d56c3ce0294720782b636595a2b3,modified,tensorflow/python/keras/mixed_precision/experi...,ff513ac54689e57ed333d62e0399a4d64a536900
...,...,...,...,...,...
88281,"@@ -342,15 +342,15 @@ def __init__(self,\n ...",23992bb091457f3e881ae1413d04c2aebbccfa2f,modified,tensorflow/contrib/seq2seq/python/ops/attentio...,af14ed3f37d52220394fb9ff902ae62fd915dbe8
88282,"@@ -755,8 +755,8 @@ def sparse_softmax_cross_e...",23992bb091457f3e881ae1413d04c2aebbccfa2f,modified,tensorflow/python/ops/losses/losses_impl.py,af14ed3f37d52220394fb9ff902ae62fd915dbe8
88283,"@@ -1646,9 +1646,9 @@ def _swap_axis(logits, d...",23992bb091457f3e881ae1413d04c2aebbccfa2f,modified,tensorflow/python/ops/nn_ops.py,af14ed3f37d52220394fb9ff902ae62fd915dbe8
88284,"@@ -369,8 +369,7 @@ def testEllipsis(self):\n ...",caceb02f75ff80a8e48440720cec3d7d6fa3297e,modified,tensorflow/python/kernel_tests/array_ops_test.py,cb9d147e9c788cc60ebb255fd26971719c7e2db2


# 构造url模板

In [8]:
contents_template = "https://api.github.com/repos/tensorflow/tensorflow/contents/{filename}?ref={sha}"

In [9]:
contents_template.format(filename=df.iloc[2]['filename'], sha=df.iloc[2]['sha'])

'https://api.github.com/repos/tensorflow/tensorflow/contents/tensorflow/python/data/experimental/kernel_tests/optimize_dataset_test.py?ref=7d9299129c30405270813fe32f1310dbf3bab265'

# 尝试获取相应内容

In [16]:
def get_html_content(url, retry=5, params=None):
    """直接获取url的内容，Exception由外部处理"""
    while retry > 0:
        try:
            r = requests.get(url, params, timeout=10)
            r.raise_for_status()
            r.encoding = r.apparent_encoding
            return r.text
        except requests.exceptions.ReadTimeout:
            logging.info("ReadTimeOut!")
            retry -= 1
        except requests.exceptions.ConnectionError:
            logging.info("ConnectionTimeOut!")
            retry -= 1
        except requests.exceptions.HTTPError as e:
            logging.info("HTTPError!, {}, {}".format(type(e), str(e)))
            retry -= 1

In [31]:
def getfile(filename, sha):
    url = contents_template.format(filename=filename, sha=sha)
    text = get_html_content(url=url)
    content = json.loads(text)
    if content['encoding'] == 'base64':
        decoded_content = base64.b64decode(content['content']).decode('utf8')
    return decoded_content

In [32]:
origin_file = getfile(filename=df.iloc[2]['filename'], sha=df.iloc[2]['sha'])
parent_file = getfile(filename=df.iloc[2]['filename'], sha=df.iloc[2]['parents_sha'])

# 获得父节点的文件

In [26]:
tempurl_2 = contents_template.format(filename=df.iloc[2]['filename'], sha=df.iloc[2]['parents_sha'])
text = get_html_content(url=tempurl_2)
content = json.loads(text)
if content['encoding'] == 'base64':
    decoded_content = base64.b64decode(content['content'])

# 比较

In [33]:
origin_file_lines = origin_file.splitlines()
parent_file_lines = parent_file.splitlines()

In [54]:
d = difflib.Differ()
diff = d.compare(parent_file_lines, origin_file_lines)

In [42]:
print ('\n'.join(list(diff)))

  # Copyright 2018 The TensorFlow Authors. All Rights Reserved.
  #
  # Licensed under the Apache License, Version 2.0 (the "License");
  # you may not use this file except in compliance with the License.
  # You may obtain a copy of the License at
  #
  #     http://www.apache.org/licenses/LICENSE-2.0
  #
  # Unless required by applicable law or agreed to in writing, software
  # distributed under the License is distributed on an "AS IS" BASIS,
  # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  # See the License for the specific language governing permissions and
  # limitations under the License.
  # ==============================================================================
  """Tests for the private `_OptimizeDataset` transformation."""
  from __future__ import absolute_import
  from __future__ import division
  from __future__ import print_function
  
  import functools
- import os
  import warnings
  
  from absl.testing import parameterized
  impor

# 查看原来变动paste，与比较结果进行比较

In [40]:
print(df.iloc[2]['patch'])

@@ -18,7 +18,6 @@
 from __future__ import print_function
 
 import functools
-import os
 import warnings
 
 from absl.testing import parameterized
@@ -213,17 +212,11 @@ def flat_map_fn(_):
                          _disable_intra_op_parallelism_test_combinations()))
   def testOptimizationDisableIntraOpParallelism(self, dataset_fn,
                                                 expected_output):
-    os.environ["TF_DATA_EXPERIMENT_OPT_IN"] = "disable_intra_op_parallelism"
-    os.environ["TF_JOB_NAME"] = "test_job"
-
     dataset = dataset_fn()
     dataset = dataset.apply(testing.assert_next(["MaxIntraOpParallelism"]))
 
     self.assertDatasetProduces(dataset, expected_output=expected_output)
 
-    del os.environ["TF_DATA_EXPERIMENT_OPT_IN"]
-    del os.environ["TF_JOB_NAME"]
-
   @combinations.generate(test_base.default_test_combinations())
   def testOptimizationThreadPoolDataset(self):
     dataset = dataset_ops.Dataset.range(10).batch(10)


# 筛选比较结果为-的行

In [55]:
temp_tesult = list(diff)

In [59]:
minus_lines = []
for line in temp_tesult:
    if line.startswith('-') and len(line) > 2: # 排除空行，因为空行只包括两个符号：减号与空格
        minus_lines.append(line.lstrip('- '))

In [60]:
minus_lines

['import os',
 'os.environ["TF_DATA_EXPERIMENT_OPT_IN"] = "disable_intra_op_parallelism"',
 'os.environ["TF_JOB_NAME"] = "test_job"',
 'del os.environ["TF_DATA_EXPERIMENT_OPT_IN"]',
 'del os.environ["TF_JOB_NAME"]']

# 遍历df

In [64]:
for index, row in df.iloc[:10].iterrows():
    print(row['patch'])

@@ -224,12 +224,7 @@ def __init__(self,
     if params:
       self.set_params(params)
 
-    self._queue_length = 10
-    self._reset_batch_timing()
-
-    # Determines if batch-level hooks need to be called.
-    # This is important for performance, because processing batch-level logs
-    # will cause async eager to block on each batch.
+    # Performance optimization: determines if batch hooks need to be called.
     # pylint: disable=protected-access
     self._should_call_train_batch_hooks = any(
         cb._implements_train_batch_hooks() for cb in self.callbacks)
@@ -239,6 +234,11 @@ def __init__(self,
         cb._implements_predict_batch_hooks() for cb in self.callbacks)
     # pylint: enable=protected-access
 
+    # Performance check: Check batch hooks for slowness compared to batch time.
+    self._timing = {}
+    self._check_timing = False
+    self._batch_start_time = None
+
   def _add_default_callbacks(self, add_history, add_progbar):
     """Adds `Callback`s that are

# 查看那些df['status']!='modify'的行正不正常

In [66]:
groups = df.groupby('status')

In [68]:
for name, group in groups:
    print(name)

('added',                                                    patch  \
3      @@ -0,0 +1,23 @@\n+# Copyright 2019 The Tensor...   
5      @@ -0,0 +1,128 @@\n+# Copyright 2019 The Tenso...   
6      @@ -0,0 +1,123 @@\n+# Copyright 2019 The Tenso...   
8      @@ -0,0 +1,58 @@\n+# Copyright 2019 The Tensor...   
11     @@ -0,0 +1,244 @@\n+# Copyright 2017 The Tenso...   
...                                                  ...   
88200  @@ -0,0 +1,559 @@\n+# Copyright 2018 The Tenso...   
88263  @@ -0,0 +1,429 @@\n+# Copyright 2018 The Tenso...   
88264  @@ -0,0 +1,286 @@\n+# Copyright 2017 The Tenso...   
88270  @@ -0,0 +1,21 @@\n+# Copyright 2016 The Tensor...   
88271  @@ -0,0 +1,50 @@\n+# Copyright 2016 The Tensor...   

                                            sha status  \
3      23ec33ae1418d56c3ce0294720782b636595a2b3  added   
5      23ec33ae1418d56c3ce0294720782b636595a2b3  added   
6      23ec33ae1418d56c3ce0294720782b636595a2b3  added   
8      23ec33ae1418d56c3ce0294720782b

In [73]:
import logging
logging.basicConfig(
    level=logging.INFO
)

## added不行，parents_sha对应的url会报错404：HTTPError

In [71]:
df_added = df[df['status']=='added']
df_added

,patch,sha,status,filename,parents_sha
3,"@@ -0,0 +1,23 @@\n+# Copyright 2019 The Tensor...",23ec33ae1418d56c3ce0294720782b636595a2b3,added,tensorflow/python/keras/mixed_precision/experi...,ff513ac54689e57ed333d62e0399a4d64a536900
5,"@@ -0,0 +1,128 @@\n+# Copyright 2019 The Tenso...",23ec33ae1418d56c3ce0294720782b636595a2b3,added,tensorflow/python/keras/mixed_precision/experi...,ff513ac54689e57ed333d62e0399a4d64a536900
6,"@@ -0,0 +1,123 @@\n+# Copyright 2019 The Tenso...",23ec33ae1418d56c3ce0294720782b636595a2b3,added,tensorflow/python/keras/mixed_precision/experi...,ff513ac54689e57ed333d62e0399a4d64a536900
8,"@@ -0,0 +1,58 @@\n+# Copyright 2019 The Tensor...",23ec33ae1418d56c3ce0294720782b636595a2b3,added,tensorflow/python/keras/mixed_precision/experi...,ff513ac54689e57ed333d62e0399a4d64a536900
11,"@@ -0,0 +1,244 @@\n+# Copyright 2017 The Tenso...",1a0742f6a7a06ff54481385b5c51094b0fef8cf3,added,tensorflow/contrib/distributions/python/kernel...,7314d0853e2ee7d64526ad3b1dcaa08591199bfb
...,...,...,...,...,...
88200,"@@ -0,0 +1,559 @@\n+# Copyright 2018 The Tenso...",6c26c995db67a70d95974fea71103712a12128bc,added,tensorflow/python/ops/ragged/row_partition_tes...,0b0432ae2d7519aa3f8f9e9569429685024fa823
88263,"@@ -0,0 +1,429 @@\n+# Copyright 2018 The Tenso...",be15ecc9c1e692b1b562d7d23f19bd1263896eef,added,tensorflow/contrib/tpu/python/tpu/feature_colu...,10ef7edc881ee715eaae48656fcb431fe128441f
88264,"@@ -0,0 +1,286 @@\n+# Copyright 2017 The Tenso...",be15ecc9c1e692b1b562d7d23f19bd1263896eef,added,tensorflow/contrib/tpu/python/tpu/feature_colu...,10ef7edc881ee715eaae48656fcb431fe128441f
88270,"@@ -0,0 +1,21 @@\n+# Copyright 2016 The Tensor...",a5e51222300c0ecf644087f99f3183644b76fd00,added,tensorflow/contrib/compiler/__init__.py,9f2bc83209c4a4f2f57697716f4509e3aa7dde8e


In [74]:
origin_file = getfile(filename=df_added.iloc[2]['filename'], sha=df_added.iloc[2]['sha'])
parent_file = getfile(filename=df_added.iloc[2]['filename'], sha=df_added.iloc[2]['parents_sha'])


origin_file_lines = origin_file.splitlines()
parent_file_lines = parent_file.splitlines()
d = difflib.Differ()
diff = d.compare(parent_file_lines, origin_file_lines)

minus_lines = []
for line in list(diff):
    if line.startswith('-') and len(line) > 2: # 排除空行，因为空行只包括两个符号：减号与空格
        minus_lines.append(line.lstrip('- '))

INFO:root:HTTPError!, <class 'requests.exceptions.HTTPError'>, 404 Client Error: Not Found for url: https://api.github.com/repos/tensorflow/tensorflow/contents/tensorflow/python/keras/mixed_precision/experimental/loss_scale_optimizer_test.py?ref=ff513ac54689e57ed333d62e0399a4d64a536900
INFO:root:HTTPError!, <class 'requests.exceptions.HTTPError'>, 404 Client Error: Not Found for url: https://api.github.com/repos/tensorflow/tensorflow/contents/tensorflow/python/keras/mixed_precision/experimental/loss_scale_optimizer_test.py?ref=ff513ac54689e57ed333d62e0399a4d64a536900
INFO:root:HTTPError!, <class 'requests.exceptions.HTTPError'>, 404 Client Error: Not Found for url: https://api.github.com/repos/tensorflow/tensorflow/contents/tensorflow/python/keras/mixed_precision/experimental/loss_scale_optimizer_test.py?ref=ff513ac54689e57ed333d62e0399a4d64a536900
INFO:root:HTTPError!, <class 'requests.exceptions.HTTPError'>, 404 Client Error: Not Found for url: https://api.github.com/repos/tensorflow

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [75]:
minus_lines

['import os',
 'os.environ["TF_DATA_EXPERIMENT_OPT_IN"] = "disable_intra_op_parallelism"',
 'os.environ["TF_JOB_NAME"] = "test_job"',
 'del os.environ["TF_DATA_EXPERIMENT_OPT_IN"]',
 'del os.environ["TF_JOB_NAME"]']

## 只有modified可以。

In [76]:
df_renamed = df[df['status']=='renamed']
df_renamed

,patch,sha,status,filename,parents_sha
436,"@@ -23,7 +23,6 @@\n from tensorflow.python.fea...",d98a0e601762228e0c0666f964530a470432bade,renamed,tensorflow/python/keras/feature_column/dense_f...,efd77d2e45f2958615a15812d225caa093f1e5af
437,"@@ -18,22 +18,25 @@\n from __future__ import d...",d98a0e601762228e0c0666f964530a470432bade,renamed,tensorflow/python/keras/feature_column/dense_f...,efd77d2e45f2958615a15812d225caa093f1e5af
438,"@@ -18,10 +18,9 @@\n from __future__ import di...",d98a0e601762228e0c0666f964530a470432bade,renamed,tensorflow/python/keras/feature_column/dense_f...,efd77d2e45f2958615a15812d225caa093f1e5af
439,"@@ -23,14 +23,14 @@\n from tensorflow.python.c...",d98a0e601762228e0c0666f964530a470432bade,renamed,tensorflow/python/keras/feature_column/dense_f...,efd77d2e45f2958615a15812d225caa093f1e5af
1024,None,dfaf790b339c5e05ad59d85bb878813c825aa301,renamed,tensorflow/python/debug/lib/debug_data.py,07bb8ea2379bd459832b23951fb20ec47f3fdbd4
...,...,...,...,...,...
87281,None,33cef2740f10e1e3791f10c8e59a9d5c8d74629d,renamed,tensorflow/contrib/training/python/training/sa...,8a3107801d15bf8af36221ff5bca0b94bf44d6d3
87282,"@@ -20,6 +20,7 @@\n \n import numpy as np\n im...",33cef2740f10e1e3791f10c8e59a9d5c8d74629d,renamed,tensorflow/contrib/training/python/training/sa...,8a3107801d15bf8af36221ff5bca0b94bf44d6d3
87283,"@@ -19,6 +19,7 @@\n from __future__ import pri...",33cef2740f10e1e3791f10c8e59a9d5c8d74629d,renamed,tensorflow/contrib/training/python/training/sa...,8a3107801d15bf8af36221ff5bca0b94bf44d6d3
87883,None,b1932a999db899743407c9d140f04e236c40fa9c,renamed,tensorflow/compiler/tests/data_format_ops_test.py,0aa06986aa4eb2fcb5b3f21cd444f55928eb52ff


In [114]:
origin_file = getfile(filename=df_renamed.iloc[3]['filename'], sha=df_renamed.iloc[3]['sha'])
parent_file = getfile(filename=df_renamed.iloc[3]['filename'], sha=df_renamed.iloc[3]['parents_sha'])


origin_file_lines = origin_file.splitlines()
parent_file_lines = parent_file.splitlines()
d = difflib.Differ()
diff = d.compare(parent_file_lines, origin_file_lines)

minus_lines = []
for line in list(diff):
    if line.startswith('-') and len(line) > 2: # 排除空行，因为空行只包括两个符号：减号与空格
        minus_lines.append(line.lstrip('- '))

INFO:root:HTTPError!, <class 'requests.exceptions.HTTPError'>, 404 Client Error: Not Found for url: https://api.github.com/repos/tensorflow/tensorflow/contents/tensorflow/python/keras/feature_column/dense_features_v2_test.py?ref=efd77d2e45f2958615a15812d225caa093f1e5af
INFO:root:HTTPError!, <class 'requests.exceptions.HTTPError'>, 404 Client Error: Not Found for url: https://api.github.com/repos/tensorflow/tensorflow/contents/tensorflow/python/keras/feature_column/dense_features_v2_test.py?ref=efd77d2e45f2958615a15812d225caa093f1e5af
INFO:root:HTTPError!, <class 'requests.exceptions.HTTPError'>, 404 Client Error: Not Found for url: https://api.github.com/repos/tensorflow/tensorflow/contents/tensorflow/python/keras/feature_column/dense_features_v2_test.py?ref=efd77d2e45f2958615a15812d225caa093f1e5af
INFO:root:HTTPError!, <class 'requests.exceptions.HTTPError'>, 404 Client Error: Not Found for url: https://api.github.com/repos/tensorflow/tensorflow/contents/tensorflow/python/keras/featur

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [ ]:
minus_lines

# 把df分成两份方便并行处理

In [79]:
df_modified = df[df['status']=='modified']
df_modified

,patch,sha,status,filename,parents_sha
0,"@@ -224,12 +224,7 @@ def __init__(self,\n ...",f21a440cde082e8b629f930c7c3f7b65ac27d4e6,modified,tensorflow/python/keras/callbacks.py,93d545e8b1e377926a24584a25c06dd422b7cc82
1,"@@ -274,6 +274,37 @@ def test_progbar_logging(...",f21a440cde082e8b629f930c7c3f7b65ac27d4e6,modified,tensorflow/python/keras/callbacks_test.py,93d545e8b1e377926a24584a25c06dd422b7cc82
2,"@@ -18,7 +18,6 @@\n from __future__ import pri...",7d9299129c30405270813fe32f1310dbf3bab265,modified,tensorflow/python/data/experimental/kernel_tes...,2c7d978a1446adaba32d8465aceef0b2c4a41cad
4,"@@ -33,7 +33,10 @@\n from tensorflow.python.ke...",23ec33ae1418d56c3ce0294720782b636595a2b3,modified,tensorflow/python/keras/mixed_precision/experi...,ff513ac54689e57ed333d62e0399a4d64a536900
7,"@@ -83,6 +83,9 @@ class Policy(object):\n ...",23ec33ae1418d56c3ce0294720782b636595a2b3,modified,tensorflow/python/keras/mixed_precision/experi...,ff513ac54689e57ed333d62e0399a4d64a536900
...,...,...,...,...,...
88281,"@@ -342,15 +342,15 @@ def __init__(self,\n ...",23992bb091457f3e881ae1413d04c2aebbccfa2f,modified,tensorflow/contrib/seq2seq/python/ops/attentio...,af14ed3f37d52220394fb9ff902ae62fd915dbe8
88282,"@@ -755,8 +755,8 @@ def sparse_softmax_cross_e...",23992bb091457f3e881ae1413d04c2aebbccfa2f,modified,tensorflow/python/ops/losses/losses_impl.py,af14ed3f37d52220394fb9ff902ae62fd915dbe8
88283,"@@ -1646,9 +1646,9 @@ def _swap_axis(logits, d...",23992bb091457f3e881ae1413d04c2aebbccfa2f,modified,tensorflow/python/ops/nn_ops.py,af14ed3f37d52220394fb9ff902ae62fd915dbe8
88284,"@@ -369,8 +369,7 @@ def testEllipsis(self):\n ...",caceb02f75ff80a8e48440720cec3d7d6fa3297e,modified,tensorflow/python/kernel_tests/array_ops_test.py,cb9d147e9c788cc60ebb255fd26971719c7e2db2


In [81]:
split_pivot = len(df_modified) // 2
split_pivot

39360

In [82]:
df_modified_part1 = df_modified[:split_pivot]
df_modified_part2 = df_modified[split_pivot:]

In [84]:
df_modified_part1.to_pickle("df_sha_part1.tar.bz2")
df_modified_part2.to_pickle("df_sha_part2.tar.bz2")

# 找出-开头的行还不行，要全部保存

In [92]:
def get_file_content(filename, sha):
    url = contents_template.format(filename=filename, sha=sha)
    text = get_html_content(url=url)
    content = json.loads(text)
    if len(content) == 0:
        return ""
    return content.get('content')

In [88]:
def get_file_parent_content(filename, sha):
    return get_file_content(filename=row[3], sha=row[4])

def get_file_origin_content(row):
    return get_file_content(filename=row[3], sha=row[1])

In [91]:
df_modified

,patch,sha,status,filename,parents_sha
0,"@@ -224,12 +224,7 @@ def __init__(self,\n ...",f21a440cde082e8b629f930c7c3f7b65ac27d4e6,modified,tensorflow/python/keras/callbacks.py,93d545e8b1e377926a24584a25c06dd422b7cc82
1,"@@ -274,6 +274,37 @@ def test_progbar_logging(...",f21a440cde082e8b629f930c7c3f7b65ac27d4e6,modified,tensorflow/python/keras/callbacks_test.py,93d545e8b1e377926a24584a25c06dd422b7cc82
2,"@@ -18,7 +18,6 @@\n from __future__ import pri...",7d9299129c30405270813fe32f1310dbf3bab265,modified,tensorflow/python/data/experimental/kernel_tes...,2c7d978a1446adaba32d8465aceef0b2c4a41cad
4,"@@ -33,7 +33,10 @@\n from tensorflow.python.ke...",23ec33ae1418d56c3ce0294720782b636595a2b3,modified,tensorflow/python/keras/mixed_precision/experi...,ff513ac54689e57ed333d62e0399a4d64a536900
7,"@@ -83,6 +83,9 @@ class Policy(object):\n ...",23ec33ae1418d56c3ce0294720782b636595a2b3,modified,tensorflow/python/keras/mixed_precision/experi...,ff513ac54689e57ed333d62e0399a4d64a536900
...,...,...,...,...,...
88281,"@@ -342,15 +342,15 @@ def __init__(self,\n ...",23992bb091457f3e881ae1413d04c2aebbccfa2f,modified,tensorflow/contrib/seq2seq/python/ops/attentio...,af14ed3f37d52220394fb9ff902ae62fd915dbe8
88282,"@@ -755,8 +755,8 @@ def sparse_softmax_cross_e...",23992bb091457f3e881ae1413d04c2aebbccfa2f,modified,tensorflow/python/ops/losses/losses_impl.py,af14ed3f37d52220394fb9ff902ae62fd915dbe8
88283,"@@ -1646,9 +1646,9 @@ def _swap_axis(logits, d...",23992bb091457f3e881ae1413d04c2aebbccfa2f,modified,tensorflow/python/ops/nn_ops.py,af14ed3f37d52220394fb9ff902ae62fd915dbe8
88284,"@@ -369,8 +369,7 @@ def testEllipsis(self):\n ...",caceb02f75ff80a8e48440720cec3d7d6fa3297e,modified,tensorflow/python/kernel_tests/array_ops_test.py,cb9d147e9c788cc60ebb255fd26971719c7e2db2


In [94]:
df_modified['origin_content'] = df_modified.apply(lambda row: get_file_content(row['filename'], row['sha']), axis=1)

KeyboardInterrupt: 

In [ ]:
df_modified['parent_content'] = df_modified.apply(lambda row: get_file_content(row['filename'], row['parents_sha']), axis=1)

# 比较文本区别，删+标记-

In [118]:
test_text = "-   # Text"
test_text[1:].strip().startswith('#')

True

In [119]:
def get_diff_file(origin_file, parent_file):
    origin_file_lines = origin_file.splitlines()
    parent_file_lines = parent_file.splitlines()
    d = difflib.Differ()
    diff = d.compare(parent_file_lines, origin_file_lines)
    minus_lines = []
    for line in list(diff):
        # 因为空行只包括两个符号：减号与空格，所以筛选len()<=2的空行
        if not line.startswith('+') and len(line) > 2:  # 新加行，或者空行都删掉，只保留原有的行与删减行
            if line.startswith('-'): # 删减行中
                if line[1:].strip().startswith('#'):  # 原来是不代码的，删掉
                    continue
                else:  # 原来是代码的，以注释代替
                    line = line.replace('-', '#', 1).strip()
            # 添加删减行和原有行，原有行原封不动
            minus_lines.append(line)
    return minus_lines

In [100]:
origin_file.splitlines()

['# Copyright 2019 The TensorFlow Authors. All Rights Reserved.',
 '#',
 '# Licensed under the Apache License, Version 2.0 (the "License");',
 '# you may not use this file except in compliance with the License.',
 '# You may obtain a copy of the License at',
 '#',
 '#     http://www.apache.org/licenses/LICENSE-2.0',
 '#',
 '# Unless required by applicable law or agreed to in writing, software',
 '# distributed under the License is distributed on an "AS IS" BASIS,',
 '# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.',
 '# See the License for the specific language governing permissions and',
 '# limitations under the License.',
 '# ==============================================================================',
 '"""A layer that produces a dense `Tensor` based on given `feature_columns`."""',
 '',
 'from __future__ import absolute_import',
 'from __future__ import division',
 'from __future__ import print_function',
 '',
 'from tensorflow.python.feature_column i

In [113]:
origin_file_lines = origin_file.splitlines()
parent_file_lines = parent_file.splitlines()
d = difflib.Differ()
diff = d.compare(parent_file_lines, origin_file_lines)
print('\n'.join(list(diff)))

- # Copyright 2018 The TensorFlow Authors. All Rights Reserved.
?                ^

+ # Copyright 2019 The TensorFlow Authors. All Rights Reserved.
?                ^

  #
  # Licensed under the Apache License, Version 2.0 (the "License");
  # you may not use this file except in compliance with the License.
  # You may obtain a copy of the License at
  #
  #     http://www.apache.org/licenses/LICENSE-2.0
  #
  # Unless required by applicable law or agreed to in writing, software
  # distributed under the License is distributed on an "AS IS" BASIS,
  # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  # See the License for the specific language governing permissions and
  # limitations under the License.
  # ==============================================================================
- """Tests for the private `_OptimizeDataset` transformation."""
+ """A layer that produces a dense `Tensor` based on given `feature_columns`."""
+ 
  from __future__ import absol

In [115]:
origin_file = getfile(filename=df_modified.iloc[3]['filename'], sha=df_modified.iloc[3]['sha'])
parent_file = getfile(filename=df_modified.iloc[3]['filename'], sha=df_modified.iloc[3]['parents_sha'])

In [122]:
print(origin_file)

# Copyright 2019 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Tests mixed precision works correctly with Keras layers and models."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl.testing import parameterized
import numpy as np

from tensorflow.python.data.ops import dataset_ops
from tensorflow.p

In [120]:
get_diff_file(origin_file, parent_file)

['  # Copyright 2019 The TensorFlow Authors. All Rights Reserved.',
 '  #',
 '  # Licensed under the Apache License, Version 2.0 (the "License");',
 '  # you may not use this file except in compliance with the License.',
 '  # You may obtain a copy of the License at',
 '  #',
 '  #     http://www.apache.org/licenses/LICENSE-2.0',
 '  #',
 '  # Unless required by applicable law or agreed to in writing, software',
 '  # distributed under the License is distributed on an "AS IS" BASIS,',
 '  # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.',
 '  # See the License for the specific language governing permissions and',
 '  # limitations under the License.',
 '  # ==============================================================================',
 '  """Tests mixed precision works correctly with Keras layers and models."""',
 '  from __future__ import absolute_import',
 '  from __future__ import division',
 '  from __future__ import print_function',
 '  from absl.testin

# 提前删docstring

In [157]:
!pip install astor -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [162]:
import ast
import astor

def delete_docstring(text):
    parsed = ast.parse(text)
    for node in ast.walk(parsed):
        # let's work only on functions & classes definitions
        if not isinstance(node, (ast.FunctionDef, ast.ClassDef, ast.AsyncFunctionDef)):
            continue

        if not len(node.body):
            continue

        if not isinstance(node.body[0], ast.Expr):
            continue

        if not hasattr(node.body[0], 'value') or not isinstance(node.body[0].value, ast.Str):
            continue

        # Uncomment lines below if you want print what and where we are removing
        print(node)
        print(node.body[0].value.s)

        node.body = node.body[1:]
        return astor.to_source(parsed)


In [163]:
print(delete_docstring(origin_file))

A layer which asserts it's inputs are a certain type.
"""Tests mixed precision works correctly with Keras layers and models."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from absl.testing import parameterized
import numpy as np
from tensorflow.python.data.ops import dataset_ops
from tensorflow.python.distribute import mirrored_strategy
from tensorflow.python.distribute import one_device_strategy
from tensorflow.python.eager import backprop
from tensorflow.python.eager import context
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import test_util
from tensorflow.python.keras import backend
from tensorflow.python.keras import layers
from tensorflow.python.keras import models
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.engine import base_layer
from tensorflow.python.keras.layers import core
from tensorflow

In [149]:
import tokenize
from io import BytesIO

def create_generator(data):
    """字符流生成器，在内部构建了一个闭包。
    为了节约内存，避免一次性加载文件内容"""

    def generator():
        for elem in data:
            try:
                yield str.encode(elem)
            except GeneratorExit:
                logging.info(f"GeneratorExit! EOF. {elem}")
                return
            except Exception as e:
                logging.info("Exception!" + str(type(e)), str(e))
                yield str.encode('')

    g = generator()  # 生成器

    def next_element():
        return next(g)

    return next_element  # 迭代器

def remove_comments_and_docstrings(source):
    """
    Returns 'source' minus comments and docstrings.
    """
#     io_obj = cStringIO.StringIO(source)
    out = ""
    prev_toktype = tokenize.INDENT
    last_lineno = -1
    last_col = 0
    
    data_generator = create_generator(source)
    
#     tokens_iterator = tokenize.generate_tokens(data_generator)
    tokens_iterator = tokenize.tokenize(BytesIO(source.encode('utf-8')).readline)
    
    for tok in tokens_iterator:
        token_type = tok[0]
        token_string = tok[1]
        start_line, start_col = tok[2]
        end_line, end_col = tok[3]
        ltext = tok[4]
        # The following two conditionals preserve indentation.
        # This is necessary because we're not using tokenize.untokenize()
        # (because it spits out code with copious amounts of oddly-placed
        # whitespace).
        if start_line > last_lineno:
            last_col = 0
        if start_col > last_col:
            out += (" " * (start_col - last_col))
        # Remove comments:
#         if token_type == tokenize.COMMENT:
#             pass
        # This series of conditionals removes docstrings:
        elif token_type == tokenize.STRING:
            if prev_toktype != tokenize.INDENT:
        # This is likely a docstring; double-check we're not inside an operator:
                if prev_toktype != tokenize.NEWLINE:
                    # Note regarding NEWLINE vs NL: The tokenize module
                    # differentiates between newlines that start a new statement
                    # and newlines inside of operators such as parens, brackes,
                    # and curly braces.  Newlines inside of operators are
                    # NEWLINE and newlines that start new code are NL.
                    # Catch whole-module docstrings:
                    if start_col > 0:
                        # Unlabelled indentation means we're inside an operator
                        out += token_string
                    # Note regarding the INDENT token: The tokenize module does
                    # not label indentation inside of an operator (parens,
                    # brackets, and curly braces) as actual indentation.
                    # For example:
                    # def foo():
                    #     "The spaces before this docstring are tokenize.INDENT"
                    #     test = [
                    #         "The spaces before this string do not get a token"
                    #     ]
        else:
            out += token_string
        prev_toktype = token_type
        last_col = end_col
        last_lineno = end_line
    return out

In [151]:
print(remove_comments_and_docstrings(origin_file))

utf-8# Copyright 2019 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

from   
from   
from   

from .testing  
import   

from .python.data.ops  
from .python.distribute  
from .python.distribute  
from .python.eager  
from .python.eager  
from .python.framework  
from .python.framework  
from .python.framework  
from .python.keras  
from .python.keras  
from .python.keras

In [144]:
origin_file.split('\n')

['# Copyright 2019 The TensorFlow Authors. All Rights Reserved.',
 '#',
 '# Licensed under the Apache License, Version 2.0 (the "License");',
 '# you may not use this file except in compliance with the License.',
 '# You may obtain a copy of the License at',
 '#',
 '#     http://www.apache.org/licenses/LICENSE-2.0',
 '#',
 '# Unless required by applicable law or agreed to in writing, software',
 '# distributed under the License is distributed on an "AS IS" BASIS,',
 '# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.',
 '# See the License for the specific language governing permissions and',
 '# limitations under the License.',
 '# ==============================================================================',
 '"""Tests mixed precision works correctly with Keras layers and models."""',
 'from __future__ import absolute_import',
 'from __future__ import division',
 'from __future__ import print_function',
 '',
 'from absl.testing import parameterized',
 'import

In [169]:
import token

def delete_comment_and_docstrings(source):
#     source = open(fname)
#     mod = open(fname + ",strip", "w")
    modified = ""

    prev_toktype = token.INDENT
    first_line = None
    last_lineno = -1
    last_col = 0

#     tokgen = tokenize.generate_tokens(source.readline)
    tokens_iterator = tokenize.tokenize(BytesIO(source.encode('utf-8')).readline)
    for toktype, ttext, (slineno, scol), (elineno, ecol), ltext in tokens_iterator:
        if 0:   # Change to if 1 to see the tokens fly by.
            print("%10s %-14s %-20r %r" % (
                tokenize.tok_name.get(toktype, toktype),
                "%d.%d-%d.%d" % (slineno, scol, elineno, ecol),
                ttext, ltext
                ))
        if slineno > last_lineno:
            last_col = 0
        if scol > last_col:
#             mod.write(" " * (scol - last_col))
            modified = modified + " " * (scol - last_col)
        if toktype == token.STRING and (prev_toktype == token.INDENT or prev_toktype == token.NEWLINE):
            # Docstring
#             mod.write("#--")
#             modified = modified + "#--"
            pass
        elif toktype == tokenize.COMMENT:
            # Comment
#             mod.write("##\n")
#             modified = modified + "##\n"
            pass
        else:
#             mod.write(ttext)
            modified = modified + ttext
        prev_toktype = toktype
        last_col = ecol
        last_lineno = elineno
    return modified

In [172]:
striped_modified_file = delete_comment_and_docstrings(origin_file)

In [173]:
striped_origin_file = delete_comment_and_docstrings(parent_file)

# 比较删除docstring和comments后的origin_file和parent_file

In [174]:
get_diff_file(striped_modified_file, striped_origin_file)

['  utf-8',
 '  """Tests mixed precision works correctly with Keras layers and models."""',
 '  from __future__ import absolute_import',
 '  from __future__ import division',
 '  from __future__ import print_function',
 '  from absl.testing import parameterized',
 '  import numpy as np',
 '  from tensorflow.python.data.ops import dataset_ops',
 '  from tensorflow.python.distribute import mirrored_strategy',
 '  from tensorflow.python.distribute import one_device_strategy',
 '  from tensorflow.python.eager import backprop',
 '  from tensorflow.python.eager import context',
 '  from tensorflow.python.framework import constant_op',
 '  from tensorflow.python.framework import dtypes',
 '  from tensorflow.python.framework import test_util',
 '  from tensorflow.python.keras import backend',
 '  from tensorflow.python.keras import layers',
 '  from tensorflow.python.keras import models',
 '  from tensorflow.python.keras import regularizers',
 '  from tensorflow.python.keras.engine import base

In [180]:
striped_modified_lines = striped_modified_file.splitlines()
striped_origin_lines = striped_origin_file.splitlines()
d = difflib.Differ()
diff = d.compare(striped_modified_lines, striped_origin_lines)
# print('\n'.join(list(diff)))

In [181]:
temp = list(diff)

In [187]:
temp_temp = []
for line in temp:
    if line.startswith('-'):
        temp_temp.append(line.replace('-', '#', 1))
    else:
        temp_temp.append(line)

In [188]:
temp_temp

['  utf-8',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  """Tests mixed precision works correctly with Keras layers and models."""',
 '  from __future__ import absolute_import',
 '  from __future__ import division',
 '  from __future__ import print_function',
 '  ',
 '  from absl.testing import parameterized',
 '  import numpy as np',
 '  ',
 '  from tensorflow.python.data.ops import dataset_ops',
 '  from tensorflow.python.distribute import mirrored_strategy',
 '  from tensorflow.python.distribute import one_device_strategy',
 '  from tensorflow.python.eager import backprop',
 '  from tensorflow.python.eager import context',
 '  from tensorflow.python.framework import constant_op',
 '  from tensorflow.python.framework import dtypes',
 '  from tensorflow.python.framework import test_util',
 '  from tensorflow.python.keras import backend',
 '  from tensorflow.python.keras import layers',
 '  from tensorflow.python.keras import models',
 

In [192]:
def get_diff_file(origin_file, modified_file):
    origin_file = delete_comment_and_docstrings(origin_file)
    modified_file = delete_comment_and_docstrings(modified_file)
    
    origin_lines = origin_file.splitlines()
    modified_lines = modified_file.splitlines()
    d = difflib.Differ()
    diff = d.compare(origin_lines, modified_lines)
    
    minus_lines = []
    for line in list(diff):
        if len(line.lstrip(' ')) == 0 or len(line.lstrip('- ')) == 0 or line.startswith('+') or line.startswith('?'):
            continue
        if line.startswith('-'):
            minus_lines.append('#' + line[2:])
        else:
            minus_lines.append(line[2:])
    return minus_lines

In [196]:
result = get_diff_file(parent_file, origin_file)
print('\n'.join(result))

utf-8
"""Tests mixed precision works correctly with Keras layers and models."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from absl.testing import parameterized
import numpy as np
from tensorflow.python.data.ops import dataset_ops
from tensorflow.python.distribute import mirrored_strategy
from tensorflow.python.distribute import one_device_strategy
from tensorflow.python.eager import backprop
from tensorflow.python.eager import context
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import test_util
from tensorflow.python.keras import backend
from tensorflow.python.keras import layers
from tensorflow.python.keras import models
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.engine import base_layer
from tensorflow.python.keras.mixed_precision.experimental import policy
from tensorflow.python.keras.optimizer_

In [194]:
get_diff_file(origin_file, parent_file)

['utf-8',
 '"""Tests mixed precision works correctly with Keras layers and models."""',
 'from __future__ import absolute_import',
 'from __future__ import division',
 'from __future__ import print_function',
 'from absl.testing import parameterized',
 'import numpy as np',
 'from tensorflow.python.data.ops import dataset_ops',
 'from tensorflow.python.distribute import mirrored_strategy',
 'from tensorflow.python.distribute import one_device_strategy',
 'from tensorflow.python.eager import backprop',
 'from tensorflow.python.eager import context',
 'from tensorflow.python.framework import constant_op',
 'from tensorflow.python.framework import dtypes',
 'from tensorflow.python.framework import test_util',
 'from tensorflow.python.keras import backend',
 'from tensorflow.python.keras import layers',
 'from tensorflow.python.keras import models',
 'from tensorflow.python.keras import regularizers',
 'from tensorflow.python.keras.engine import base_layer',
 '#from tensorflow.python.keras